In [374]:
import tensorflow as tf

In [406]:
import math
import numpy as np

In [376]:
def arctanh(x):
    return tf.log(tf.divide(1+x,1-x))

Initialise all of the variables

In [377]:
def inner_prod(r_in, r_out, theta_in, theta_out):
    cosine = tf.cos(theta_in - theta_out)
    radius = tf.multiply(arctanh(r_in), arctanh(r_out))
    return 4 * tf.multiply(cosine, radius)

In [378]:
def minkowski_dot(u,v):
    return tf.tensordot(u,v,1) - 2*tf.multiply(u[0],v[0])

In [379]:
def exponential(base, tangent):
    """
    Compute the exponential of `tangent` from the point `base`.
    """
    #tangent = tangent.copy()
    norm = tf.sqrt(tf.maximum(minkowski_dot(tangent, tangent), 0))
    if norm == 0:
        return base
    tangent /= norm
    return tf.cosh(norm) * base + tf.sinh(norm) * tangent

In [380]:
def tensor_inner_prod(r_example, r_sample, theta_example, theta_sample):
    r1 = arctanh(r_example)
    r2 = arctanh(r_sample)
    radius_term = r1[:, None] + r2[None, :]
    cos_term = theta_example[:, None] - theta_sample[None, :]
    return tf.squeeze(4* tf.multiply(cos_term, radius_term))

In [381]:
def nce_loss(true_logits, sampled_logits):
        true_xent = tf.nn.sigmoid_cross_entropy_with_logits(
            labels=tf.ones_like(true_logits), logits=true_logits)
        sampled_xent = tf.nn.sigmoid_cross_entropy_with_logits(
            labels=tf.zeros_like(sampled_logits), logits=sampled_logits)
        nce_loss_tensor = (tf.reduce_sum(true_xent) +
                           tf.reduce_sum(sampled_xent)) / 2
        return nce_loss_tensor

In [382]:
radius_in = tf.Variable(tf.sqrt(tf.random_uniform([5,1])))  # radius
# radius_out = tf.Variable(tf.zeros([5,1]))
radius_out = tf.Variable(tf.sqrt(tf.random_uniform([5,1])))
theta_in = tf.Variable(2*np.pi*tf.random_uniform([5,1]))  # angle
theta_out = tf.Variable(2*np.pi*tf.random_uniform([5,1]))
# theta_out = tf.Variable(tf.zeros([5,1]))
sm_b = tf.Variable(tf.zeros([5,1]))
examples = tf.Variable([1.0,2.0])
labels = tf.Variable([3.0,4.0])

In [383]:
example_radius = tf.nn.embedding_lookup(radius_in, examples)
example_theta = tf.nn.embedding_lookup(theta_in, examples)
true_radius = tf.nn.embedding_lookup(radius_out, labels)
true_theta = tf.nn.embedding_lookup(theta_out, labels)

TypeError: Value passed to parameter 'indices' has DataType float32 not in list of allowed values: int32, int64

In [384]:
sampled_ids = tf.Variable([0,1,2])
sampled_radius = tf.nn.embedding_lookup(radius_out, sampled_ids)
sampled_theta = tf.nn.embedding_lookup(theta_out, sampled_ids)
true_b = tf.nn.embedding_lookup(sm_b, labels)

TypeError: Value passed to parameter 'indices' has DataType float32 not in list of allowed values: int32, int64

In [ ]:
true_logits = inner_prod(example_radius, true_radius, example_theta, true_theta) + true_b

In [385]:
sampled_b = tf.nn.embedding_lookup(sm_b, sampled_ids)
sampled_b_vec = tf.reshape(sampled_b, sampled_ids.get_shape())

In [386]:
sampled_logits = tensor_inner_prod(example_radius, sampled_radius, example_theta, sampled_theta) + sampled_b_vec

NameError: name 'example_radius' is not defined

In [387]:
r1 = arctanh(example_radius)
r2 = arctanh(sampled_radius)
radius_term = r1[:, None] + r2[None, :]
cos_term = example_theta[:, None] - sampled_theta[None, :]
retval = 4* tf.multiply(cos_term, radius_term)

NameError: name 'example_radius' is not defined

In [388]:
loss = nce_loss(true_logits, sampled_logits)

NameError: name 'true_logits' is not defined

In [389]:
def minkowski_dist(u, v):
    """
    The distance between two points in Minkowski space
    :param u:
    :param v:
    :return:
    """
    return tf.acosh(-minkowski_dot(u, v))

In [390]:
def project_onto_tangent_space(hyperboloid_point, minkowski_tangent):
    """
    project gradients in the ambiant space onto the tangent space
    :param hyperboloid_point:
    :param minkowski_tangent:
    :return:
    """
    return minkowski_tangent + minkowski_dot(hyperboloid_point, minkowski_tangent) * hyperboloid_point

In [391]:
def exp_map(base, tangent):
    """
    Compute the exponential of the `tangent` vector from the point `base`.
    """
    # tangent = tangent.copy()
    norm = tf.sqrt(tf.maximum(minkowski_dot(tangent, tangent), 0))
    if norm == 0:
        return base
    tangent /= norm
    return tf.cosh(norm) * base + tf.sinh(norm) * tangent

In [392]:
grads = np.array([[1., 2., 0.2], [3., 4., 0.5], [4., 2., 0.6]])
x = np.eye(grads.shape[1])
x[0,0]=-1
grads.dot(x)

array([[-1. ,  2. ,  0.2],
       [-3. ,  4. ,  0.5],
       [-4. ,  2. ,  0.6]])

In [393]:
# V = tf.Variable(np.array([[1,2],[4,5],[5,2]]), dtype=tf.int32)
V = tf.Variable(np.array([1,2]), dtype=tf.int32)
x = np.eye(V.shape[0])
x[0,0]=-1
T = tf.constant(x, dtype=tf.int32)
# U = tf.matmul(V,T)
U = tf.tensordot(V,T, 1)
# y = tf.constant([math.sqrt(10001),100])
y = tf.constant([1.,0])
z = tf.constant([1.,0])
g1 = tf.constant([0.,1.])
g2 = tf.constant([0.,-1.])
g3 = tf.constant([0.,-2.])
v = tf.constant([1.,1.])
s = tf.constant([math.sqrt(5), 2])
t = tf.constant([6.2, 3.2])

In [394]:
grads = tf.Variable(np.array([[1., 2., 0.2], [3., 4., 0.5], [4., 2., 0.6]]))
true_val = np.array([[-1., 2., 0.2], [-3., 4., 0.5], [-4., 2., 0.6]])
x = np.eye(grads.shape[1])
x[0, 0] = -1.
T = tf.constant(x, dtype=tf.float64)

In [395]:
def minkowski_tensor_dot(u, v):
    """
    Minkowski dot product is the same as the Euclidean dot product, but the first element squared is subtracted
    :param u: a tensor of shape (#examples, dims)
    :param v: a tensor of shape (#examples, dims)
    :return: a scalar dot product
    """
    assert u.shape == v.shape, 'minkowski dot product not define for different shape tensors'
    try:
        temp = np.eye(u.shape[1])
    except IndexError:
        temp = np.eye(u.shape)
    temp[0, 0] = -1.
    T = tf.constant(temp, dtype=u.dtype)
    # make the first column of v negative
    v_neg = tf.matmul(v, T)
    return tf.reduce_sum(tf.multiply(u, v_neg), 1, keep_dims=True)  # keep dims for broadcasting

In [396]:
def tensor_exp_map(hyperboloid_points, tangent_grads):
    """
    Map vectors in the tangent space of the hyperboloid points back onto the hyperboloid
    :param hyperboloid_points: a tensor of points on the hyperboloid of shape (#examples, #dims)
    :param tangent_grads: a tensor of gradients on the tangent spaces of the hyperboloid_points of shape (#examples, #dims)
    :return:
    """
    # todo do we need to normalise the gradients?
    norms = tf.sqrt(tf.maximum(minkowski_tensor_dot(tangent_grads, tangent_grads), 0))
    zero = tf.constant(0, dtype=tf.float32)
    nonzero_flags = tf.squeeze(tf.not_equal(norms, zero))
    nonzero_indices = tf.squeeze(tf.where(nonzero_flags))
    nonzero_norms = tf.boolean_mask(norms, nonzero_flags)
    updated_grads = tf.boolean_mask(tangent_grads, tf.squeeze(nonzero_flags))
    updated_points = tf.boolean_mask(hyperboloid_points, nonzero_flags)
    # if norms == 0:
    #     return hyperboloid_points
    normed_grads = tf.divide(updated_grads, nonzero_norms)
    updates = tf.multiply(tf.cosh(nonzero_norms), updated_points) + tf.multiply(tf.sinh(nonzero_norms), normed_grads)
    return tf.scatter_update(hyperboloid_points, nonzero_indices, updates)

In [397]:
def project_tensors_onto_tangent_space(hyperboloid_points, ambient_gradients):
    """
    project gradients in the ambiant space onto the tangent space
    :param hyperboloid_point: A point on the hyperboloid
    :param ambient_gradient: The gradient to project
    :return: gradients in the tangent spaces of the hyperboloid points
    """
    return ambient_gradients + tf.multiply(minkowski_tensor_dot(hyperboloid_points, ambient_gradients),
                                           hyperboloid_points)

In [398]:
def transform_grads(grad):
    """
    multiply by the inverse of the Minkowski metric tensor g = diag[-1, 1,1 ... 1] to make the first element of each
    grad vector negative
    :param grad: grad matrix of shape (n_vars, embedding_dim)
    :return:
    """
    x = np.eye(grad.shape[1])
    x[0, 0] = -1.
    T = tf.constant(x, dtype=grad.dtype)
    return tf.matmul(grad, T)

In [399]:
def rsgd(grads, vecs, lr=0.1):
    """
    Perform the Riemannian gradient descent operation by
    1/ Transforming gradients using the Minkowski metric tensor
    2/ Projecting onto the tangent space
    3/ Applying the exponential map
    :param grads:
    :param var:
    :param lr:
    :return:
    """
    minkowski_grads = transform_grads(grads)
    tangent_grads = project_tensors_onto_tangent_space(vecs, minkowski_grads)
    return tensor_exp_map(vecs, lr * tangent_grads)

In [400]:
g1 = tf.constant([[1., 1.], [2., -1.], [3., 2.], [4., 0.]])
retval1 = np.array([[-1., 1.], [-2., -1.], [-3., 2.], [-4., 0.]])
transformed_grads = transform_grads(g1)
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
tgs = sess.run(transformed_grads)
print(tgs)
assert np.array_equal(tgs, retval1)

[[-1.  1.]
 [-2. -1.]
 [-3.  2.]
 [-4.  0.]]


In [401]:
p1 = tf.Variable([[1., 0.], [1., 0.], [1., 0.], [1., 0.]])  # this the minima of the hyperboloid
g1 = tf.constant([[1., 1.], [2., -1.], [3., 2.], [4., 0.]])
retval1 = np.array([[-1.], [-1.], [-1.], [-1.]])
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
print(sess.run(p1))
p1 = g1
print(sess.run(p1))

[[ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]]
[[ 1.  1.]
 [ 2. -1.]
 [ 3.  2.]
 [ 4.  0.]]


In [402]:
p1 = tf.Variable([[1., 0.], [1., 0.], [1., 0.], [1., 0.]])  # this the minima of the hyperboloid
p2 = tf.Variable([[ 1.54308057, 1.17520118],[ 1.54308057,-1.17520118],[ 3.76219535,3.62686038],[ 1.,0.]])
g1 = tf.constant([[1., 1.], [2., -1.], [3., 2.], [4., 0.]])
retval1 = np.array([[-1.], [-1.], [-1.], [-1.]])
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
# here the tangent space is x=1
# print(sess.run(rsgd(g1, p1)))
minkowski_grads = transform_grads(g1)
tangent_grads = project_tensors_onto_tangent_space(p2, minkowski_grads)
pnew = tensor_exp_map(p2, tangent_grads)
# # check that the points are on the hyperboloid
# # print(sess.run(p2))
print('minkowski grads', sess.run(minkowski_grads))
print('tangent space grads', sess.run(tangent_grads))
new_points = sess.run(pnew)
print('new points', new_points)
# norms = sess.run(minkowski_tensor_dot(pnew, pnew))
norms = sess.run(minkowski_tensor_dot(new_points, new_points))
print(norms)
assert np.array_equal(np.around(norms, 3), retval1)

('minkowski grads', array([[-1.,  1.],
       [-2., -1.],
       [-3.,  2.],
       [-4.,  0.]], dtype=float32))
('tangent space grads', array([[  3.19452763,   4.19452763],
       [  4.57562494,  -6.00795794],
       [ 66.75225067,  69.24310303],
       [  0.        ,   0.        ]], dtype=float32))
('new points', array([[  2.06089172e+01,   2.05846424e+01],
       [  6.67124863e+01,  -6.67050018e+01],
       [  3.63710016e+08,   3.63710048e+08],
       [  1.00000000e+00,   0.00000000e+00]], dtype=float32))
[[ -9.99969482e-01]
 [ -9.98535156e-01]
 [  2.57698038e+10]
 [ -1.00000000e+00]]


AssertionError: 

In [ ]:
def minkowski_vector_dot(u, v):
    """
        Minkowski dot product is the same as the Euclidean dot product, but the first element squared is subtracted
        :param u: a vector
        :param v: a vector
        :return: a scalar dot product
        """
    assert u.shape == v.shape, 'minkowski dot product not define for different shape vectors'
    # assert that the vectors have only 1d.
    # todo this currently fails because exp_map returns tensors with shape = None
    # assert u.get_shape().ndims == 1, 'applied minkowski_vector_dot to a tensor. Try using minkowski_tensor_dot'

    return tf.tensordot(u, v, 1) - 2 * tf.multiply(u[0], v[0])

In [ ]:
def project_onto_tangent_space(hyperboloid_point, ambient_gradient):
    """
    project gradients in the ambiant space onto the tangent space
    :param hyperboloid_point: A point on the hyperboloid
    :param ambient_gradient: The gradient to project
    :return:
    """
    return ambient_gradient + minkowski_vector_dot(hyperboloid_point, ambient_gradient) * hyperboloid_point

In [ ]:
point = tf.Variable([3.76219535,3.62686038])
g1 = tf.constant([-3., 2.])
# minkowski_grads = transform_grads(g1)
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
print(sess.run(project_onto_tangent_space(point, g1)))

In [ ]:
points = tf.Variable([[1., 0.], [1., 0.], [1., 0.], [1., 0.]])  # this the minima of the hyperboloid
grads = tf.Variable([[1., 1.], [2., -1.], [3., 2.], [4., 0.]])
retval1 = np.array([[-1.], [-1.], [-1.], [-1.]])
sess = tf.Session()
lr = 0.1
init = tf.global_variables_initializer()
sess.run(init)
vals = []
for i in range(3):
    vals.append(sess.run(points))
    print(vals[i])
    print(sess.run(minkowski_tensor_dot(vals[i], vals[i])))
#     print(sess.run(points))
    points = rsgd(grads, points)
#     vals.append(points)
    # check that the points are on the hyperboloid
#     norms = sess.run(minkowski_tensor_dot(points, points))
#     print(norms)
#     try:
#         assert np.array_equal(np.around(norms, 3), retval1)
#     except AssertionError:
#         print(sess.run(points))
print(sess.run(vals))

In [427]:
def circ_sample():
    from matplotlib.pyplot import scatter
    # radius of the circle
    circle_r = 1
    # center of the circle (x, y)
    circle_x = 0
    circle_y = 0

    # random angle
    alpha = 2 * math.pi * np.random.rand(1000)
    # random radius
    r = circle_r * np.sqrt(np.random.rand(1000))
    # calculating coordinates
    x = r * np.cos(alpha) + circle_x
    y = r * np.sin(alpha) + circle_y
#     scatter(x,y)
    retval = np.concatenate((x,y), axis=0)
    print(retval)
    return retval

In [428]:
circ_sample()

[ 0.11786285 -0.48487972  0.73233298 ...,  0.67630736  0.06704782
 -0.45228546]


array([ 0.11786285, -0.48487972,  0.73233298, ...,  0.67630736,
        0.06704782, -0.45228546])

In [408]:
np.random.rand(10)

array([ 0.99220344,  0.47888188,  0.96114238,  0.51419669,  0.13342885,
        0.3481402 ,  0.49727209,  0.50193905,  0.44276913,  0.1926202 ])

In [446]:
def to_hyperboloid_points(poincare_pts):
    """
    Post: result.shape[1] == poincare_pts.shape[1] + 1
    """
    norm_sqd = (poincare_pts ** 2).sum(axis=1)
    N = poincare_pts.shape[1]
    result = np.zeros((poincare_pts.shape[0], N + 1), dtype=np.float64)
    result[:, 1:] = (2. / (1 - norm_sqd))[:, np.newaxis] * poincare_pts
    result[:, 0] = (1 + norm_sqd) / (1 - norm_sqd)
    return result

In [447]:
points = np.divide(np.random.rand(100,2),np.sqrt(2))

In [448]:
hyp_points = to_hyperboloid_points(points)

In [449]:
hyp_points

array([[  3.08603677e+00,   9.68986464e-01,   2.75403126e+00],
       [  1.36300074e+00,   7.86176826e-01,   4.89588628e-01],
       [  3.20336079e+00,   2.20962623e+00,   2.09262330e+00],
       [  1.47619865e+00,   1.04260346e+00,   3.03546497e-01],
       [  1.27981685e+00,   6.33842268e-01,   4.85978543e-01],
       [  3.36952527e+00,   1.32147588e+00,   2.93383743e+00],
       [  3.39460041e+00,   1.90462314e+00,   2.62597079e+00],
       [  1.14379844e+00,   1.32912895e-01,   5.39081655e-01],
       [  8.67314828e+00,   6.08228655e+00,   6.10158106e+00],
       [  1.06980832e+00,   3.58919025e-01,   1.25167802e-01],
       [  2.16135982e+00,   3.78978614e-01,   1.87825756e+00],
       [  1.37954551e+00,   9.50308411e-01,   7.72968048e-03],
       [  1.27782878e+00,   4.06931665e-01,   6.83559074e-01],
       [  2.97189045e+00,   1.72008486e+00,   2.20758713e+00],
       [  2.23136668e+00,   1.10528308e+00,   1.66052600e+00],
       [  2.79282061e+00,   9.47224276e-01,   2.4295294

In [454]:
sess = tf.Session()
lr = 0.1
init = tf.global_variables_initializer()
sess.run(init)
assert np.array_equal(np.around(sess.run(minkowski_tensor_dot(hyp_points, hyp_points)),, np.array(100 * [[-1.]]))

AssertionError: 

In [453]:
np.array(100*[[-1]])

array([[-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],